In [ ]:
!git clone https://github.com/mlsquare/mergekit-mamba.git

In [11]:
ls


/bin/bash: /home/yashc/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
codeparrot-ds/           modeling_mamba.py           requirements.txt  wandb/
configuration_mamba.py   model_parameters.json       Seshu/            wts/
encoded_inputs_mamba.pt  model_parameters_lora.json  trained_model/
LICENSE                  new_file.yaml               trial/
mamba/                   __pycache__/                trying/


In [12]:
import wandb

In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer,TrainingArguments
from huggingface_hub import HfApi, ModelFilter
import torch
from datasets import load_dataset, Dataset, DatasetDict
import json
from configuration_mamba import MambaConfig
from modeling_mamba import MambaModel, MambaForCausalLM
import torch.nn as nn
import torch.nn.functional as F
from peft import LoraConfig, TaskType, get_peft_model, PeftMixedModel
from transformers import DataCollatorForLanguageModeling, AutoModelForCausalLM
import os


In [4]:
from huggingface_hub import HfApi, ModelFilter
from peft import PeftMixedModel
def get_models_by_organization(org_id):
    api = HfApi()
    new_filter = ModelFilter(tags="mamba")
    models = api.list_models(filter=new_filter)
    models_list = []
    for i in models:
        print(i.modelId)
        if org_id in i.modelId:
            models_list.append(i.modelId)
    return models_list


org_id = "mlsquare"
models = get_models_by_organization(org_id)
models


huseinzol05/dummy-mamba-1.4b
mesolitica/mamba-1.4B-4096
Q-bert/Mamba-130M
Q-bert/Mamba-370M
Q-bert/Mamba-790M
Q-bert/Mamba-1B
Q-bert/Mamba-3B
s3nh/mamba-1.4b_dolly_instruction_polish
ybelkada/test-axolotl-axolotlmambatrainer
Q-bert/Mamba-3B-slimpj
Q-bert/MambaHermes-3B
ZySec-AI/Mamba-2.8B-CyberSec
ayoubkirouane/Mamba-Chat-2.8B
DeepMount00/Mamba-QA-ITA
kuotient/mamba-ko-2.8b
Trelis/mamba-2.8b-slimpj-bf16
mjschock/mamba-130m
mescarda/my_awesome_model
DeepMount00/Mamba-QA-ITA-790m
mjschock/mamba-370m
mjschock/mamba-790m
mjschock/mamba-1.4b
ArthurZ/small-model
ArthurZ/mamba-130m
mlsquare/exp_causal_mamba
mlsquare/mamba1
mlsquare/mamba2
mlsquare/mamba3
ArthurZ/mamba-2.8b
ArthurZ/mamba-370m
ArthurZ/mamba-790m
ArthurZ/mamba-1.4b
ArthurZ/mamba-2.8b-slimpj
mjschock/mamba-130m-ppo
mlsquare/pico_mamba
mlsquare/pico_seshu


['mlsquare/exp_causal_mamba',
 'mlsquare/mamba1',
 'mlsquare/mamba2',
 'mlsquare/mamba3',
 'mlsquare/pico_mamba',
 'mlsquare/pico_seshu']

In [106]:
models = {
    "small" : ["mlsquare/mamba_130M_small_out_proj", "mlsquare/mamba_130M_small_dt_proj", "mlsquare/mamba_130M_small_x_proj"],
    "large" : ["mlsquare/mamba_130M_large_x_dt_out_proj"]
}

def compute_loss(model, inputs, return_outputs=False): 
    lm_logits = model(inputs)[0]
    labels = inputs.to(lm_logits.device)

    shift_logits = lm_logits[:, :-1, :].contiguous()
    labels = labels[:, 1:].contiguous()
    loss_fct = torch.nn.CrossEntropyLoss()
    lm_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), labels.view(-1))
    return lm_loss

    
def evaluation(data, model, tokenizer):
    val = 0
    for i in data:
        value = tokenizer.encode(i['tgt'], return_tensors="pt")
        val += compute_loss(model, value)
        
    avg_loss = val / len(data)    
    return avg_loss

def model_merge_large(adapters, model_path, data, tokenizer):
    
    model = MambaForCausalLM.from_pretrained(model_path)
    model.load_adapter(adapters["large"][0])
    result = evaluation(data, model, tokenizer)
    return result
    
def model_merge_small(adapters, model_path, data, tokenizer):

    base_model = MambaForCausalLM.from_pretrained(model_path)
    peft_model = PeftMixedModel.from_pretrained(base_model, adapters["small"][0])
    peft_model.load_adapter(adapters["small"][1], adapter_name="1")
    peft_model.load_adapter(adapters["small"][2], adapter_name="2")
    peft_model.set_adapter(["default", "1", "2"])
    result = evaluation(data, model, tokenizer)
    return result
    
def create_JSON(value):
    json_data = json.dumps(value, indent=4)
    with open(f"{value}", "w") as json_file:
        json_file.write(json_data)
        
def get_data(data_path, fraction = 0.1):
    data = load_dataset(data_path)['train'].shuffle()
    data = data.select(list(range(int(len(data) * fraction))))
    return data
    

In [ ]:
<model>-<PARAMS>-<AdapterComputation>-<target_modules>

In [ ]:
mamba_130M_small_out_proj = {
    "model_path" : "Q-bert/Mamba-130M",
    "tokenizer_path": "Q-bert/Mamba-130M",
    "target_modules": ["out_proj"],
    "adapter_path" : "mlsquare/mamba-130M-small-out_proj",
    "data" : "mlsquare/samantar1per_cent_merged_with_train_val"
}

In [ ]:
mamba_130M_small_d_proj = {
    "model_path" : "Q-bert/Mamba-130M",
    "tokenizer_path": "Q-bert/Mamba-130M",
    "target_modules": ["d_proj"],
    "adapter_path" : "mlsquare/mamba-130M-small-out_proj",
    "data" : "mlsquare/samantar1per_cent_merged_with_train_val"
}

In [ ]:
mamba_130M_small_x_proj = {
    "model_path" : "Q-bert/Mamba-130M",
    "tokenizer_path": "Q-bert/Mamba-130M",
    "target_modules": ["x_proj"],
    "adapter_path" : "mlsquare/mamba-130M-small-out_proj",
    "data" : "mlsquare/samantar1per_cent_merged_with_train_val"
}

In [ ]:
mamba_130M_large_x_d_out_proj = {
    "model_path" : "Q-bert/Mamba-130M",
    "tokenizer_path": "Q-bert/Mamba-130M",
    "target_modules": ["x_proj", "d_proj","out_proj"],
    "adapter_path" : "mlsquare/mamba-130M-small-out_proj",
    "data" : "mlsquare/samantar1per_cent_merged_with_train_val"
}